In [1]:
import pandas as pd
import numpy as np
import datetime 
import os 

In [2]:
time_series = pd.read_csv("../../databases/JHU/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv")
time_series.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,145,145,145,145,145,149,149,151,151,151
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,14,14,14,14,14,15,15,15,16,16
3,630,PR,PRI,630,72.0,NaN,Puerto Rico,US,18.2208,-66.5901,...,1575,1757,1808,1843,1924,1968,2031,2156,2173,2198
4,850,VI,VIR,850,78.0,NaN,Virgin Islands,US,18.3358,-64.8963,...,66,66,66,66,66,66,66,68,68,69


In [3]:
time_series = time_series.dropna(subset = ["Admin2"])
time_series.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20
5,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,42,45,48,53,53,58,61,67,68,74
6,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,175,181,187,188,189,196,205,208,216,222
7,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,42,43,45,45,47,47,51,53,58,59
8,84001007,US,USA,840,1007.0,Bibb,Alabama,US,32.996421,-87.125115,...,42,42,43,42,43,43,44,44,45,46
9,84001009,US,USA,840,1009.0,Blount,Alabama,US,33.982109,-86.567906,...,39,40,40,40,40,42,44,44,44,44


In [4]:
sites = pd.read_excel("../clean_data/sites.xlsx")
sites.head()

,Site,Region,County,State,Combined_Key
0,Point Beach,3,Manitowoc,Wisconsin,"Manitowoc, Wisconsin, US"
1,Point Beach,3,Kewaunee,Wisconsin,"Kewaunee, Wisconsin, US"


In [5]:
time_series_sites = time_series.merge(sites, left_on = "Combined_Key", right_on = "Combined_Key", how = "inner")
time_series_sites.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,Site,Region,County,State
0,84055061,US,USA,840,55061.0,Kewaunee,Wisconsin,US,44.517275,-87.614684,...,22,25,25,26,28,29,Point Beach,3,Kewaunee,Wisconsin
1,84055071,US,USA,840,55071.0,Manitowoc,Wisconsin,US,44.119468,-87.809289,...,16,17,17,17,19,19,Point Beach,3,Manitowoc,Wisconsin


In [90]:
#time_series_sites = time_series.merge(sites, left_on = "Combined_Key", right_on = "Combined_Key", how = "left")
#time_series_sites.head()

In [91]:
#time_series_sites = time_series_sites.dropna(subset = ["Site"])
#time_series_sites.head()

In [6]:
counties_sites = time_series_sites[["FIPS", "Site", "Region", "County","State","Combined_Key"]]
counties_sites.head()

,FIPS,Site,Region,County,State,Combined_Key
0,55061.0,Point Beach,3,Kewaunee,Wisconsin,"Kewaunee, Wisconsin, US"
1,55071.0,Point Beach,3,Manitowoc,Wisconsin,"Manitowoc, Wisconsin, US"


In [7]:
startdate = datetime.datetime(2020,3,4)
today = datetime.datetime.today()
offset = 53 + (today-startdate).days
offset

121

In [8]:
dates = time_series_sites.iloc[:,53:offset]
dates.head()

,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,...,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20
0,0,0,0,0,0,0,0,0,0,0,...,13,15,18,22,22,25,25,26,28,29
1,0,0,0,0,0,0,0,0,0,0,...,11,13,13,14,16,17,17,17,19,19


In [9]:
site_time_series = pd.DataFrame(columns = ["FIPS","Site","Region","County","State","Combined_Key","Confirmed","Date"])

In [10]:
#county_time = pd.DataFrame(columns = ["Confirmed"])
cols = dates.columns

for i in range(14,len(cols)):
    one_day = pd.DataFrame(columns = ["Confirmed", "Date"])
    one_day["Confirmed"] = dates.iloc[:,i]
    one_day["Date"] = cols[i]
    county_time = counties_sites.join(one_day)
    site_time_series = site_time_series.append(county_time)
#    county_time.append(one_day)
#    print(one_day)
site_time_series.head()


,FIPS,Site,Region,County,State,Combined_Key,Confirmed,Date
0,55061.0,Point Beach,3,Kewaunee,Wisconsin,"Kewaunee, Wisconsin, US",0,3/18/20
1,55071.0,Point Beach,3,Manitowoc,Wisconsin,"Manitowoc, Wisconsin, US",0,3/18/20
0,55061.0,Point Beach,3,Kewaunee,Wisconsin,"Kewaunee, Wisconsin, US",0,3/19/20
1,55071.0,Point Beach,3,Manitowoc,Wisconsin,"Manitowoc, Wisconsin, US",0,3/19/20
0,55061.0,Point Beach,3,Kewaunee,Wisconsin,"Kewaunee, Wisconsin, US",0,3/20/20


In [11]:
site_time_series.reset_index(inplace = True)
site_time_series.drop("index", axis = 1, inplace = True)

In [12]:
site_time_series["Date"] = pd.to_datetime(site_time_series.Date)
site_time_series.dtypes

FIPS                   float64
Site                    object
Region                  object
County                  object
State                   object
Combined_Key            object
Confirmed               object
Date            datetime64[ns]
dtype: object

In [13]:
site_time_series["Date"] = site_time_series["Date"].dt.to_period("D")

In [14]:
site_time_series.tail()

,FIPS,Site,Region,County,State,Combined_Key,Confirmed,Date
103,55071.0,Point Beach,3,Manitowoc,Wisconsin,"Manitowoc, Wisconsin, US",17,2020-05-08
104,55061.0,Point Beach,3,Kewaunee,Wisconsin,"Kewaunee, Wisconsin, US",28,2020-05-09
105,55071.0,Point Beach,3,Manitowoc,Wisconsin,"Manitowoc, Wisconsin, US",19,2020-05-09
106,55061.0,Point Beach,3,Kewaunee,Wisconsin,"Kewaunee, Wisconsin, US",29,2020-05-10
107,55071.0,Point Beach,3,Manitowoc,Wisconsin,"Manitowoc, Wisconsin, US",19,2020-05-10


In [15]:
site_time_series = site_time_series[["FIPS", "Site", "Region", "County", "State", "Combined_Key", "Confirmed", "Date"]]
site_time_series.tail()

,FIPS,Site,Region,County,State,Combined_Key,Confirmed,Date
103,55071.0,Point Beach,3,Manitowoc,Wisconsin,"Manitowoc, Wisconsin, US",17,2020-05-08
104,55061.0,Point Beach,3,Kewaunee,Wisconsin,"Kewaunee, Wisconsin, US",28,2020-05-09
105,55071.0,Point Beach,3,Manitowoc,Wisconsin,"Manitowoc, Wisconsin, US",19,2020-05-09
106,55061.0,Point Beach,3,Kewaunee,Wisconsin,"Kewaunee, Wisconsin, US",29,2020-05-10
107,55071.0,Point Beach,3,Manitowoc,Wisconsin,"Manitowoc, Wisconsin, US",19,2020-05-10


In [16]:
site_time_series.to_excel("../clean_data/site_time_series.xlsx")